# H₂ Molecule Homework Assignment
### Quantum Software Development Journey: From Theory to Application with Classiq - Part 3

- Similarly to what we have done in class, in this exercise we will implement the VQE on H2 molecule.
- This time instead of using the built-in methods and functions (such as `Molecule` and `MoleculeProblem`) to define and solve the problem, you will be provided with a two qubits Hamiltonian.

## Submission
- Submit the completed Jupyter notebook and report via GitHub. Ensure all files are correctly named and organized.
- Use the Typeform link provided in the submission folder to confirm your submission.

## Additional Resources
- [Classiq Documentation](https://docs.classiq.io/latest/)
- The notebook from live session #3

## Important Dates
- **Assignment Release:** 22.5.2024
- **Submission Deadline:** 3.6.2024 (7 A.M GMT+3)

---

Happy coding and good luck!

### Part 1

Given the following Hamiltonian:

$$
\hat{H} = -1.0523 \cdot (I \otimes I) + 0.3979 \cdot (I \otimes Z) - 0.3979 \cdot (Z \otimes I) - 0.0112 \cdot (Z \otimes Z) + 0.1809 \cdot (X \otimes X)
$$

Complete the following code

In [1]:
from typing import List
from classiq import *

HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm],
                        [PauliTerm([Pauli.I, Pauli.I], -1.0523),
                         PauliTerm([Pauli.I, Pauli.Z],  0.3979),
                         PauliTerm([Pauli.Z, Pauli.I], -0.3979),
                         PauliTerm([Pauli.Z, Pauli.Z], -0.0112),
                         PauliTerm([Pauli.X, Pauli.X],  0.1809)]
                        ) #TODO: Complete Hamiltonian

In [2]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have arbitrary rotation
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])


@cfunc
def cmain() -> None:
    res = vqe(
        # TODO: complete the missing argument
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

#TODO: complete the line, use classical_execution_function
qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/490774a2-4a81-468f-b28c-680cd171ca48?version=0.42.0


In [3]:
execution = execute(qprog)
res = execution.result()
execution.open_in_ide()
vqe_result = res[0].value #TODO: complete the line

In [5]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.07021669921875
Optimal parameters: {'angles_0': -2.7719312634977746, 'angles_1': 4.446680549285845, 'angles_2': 3.477513433640137}
Eigenstate: {'00': (0.05412658773652741+0j), '10': (0.18221724671391565+0j), '01': (0.19008632907181935+0j), '11': (0.9631896879639026+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class?

### Part 2

**Now, we want to have a more interesting ansatz in our `main`.**  
Add **one** line of code to the `main` function you created in Part 1 that creates **entanglement** between the two qubits.  
Which gate should you use?

In [6]:
@qfunc
def main(q: Output[QArray[QBit]], angles: CArray[CReal, 3]) -> None:
    # TODO: Create an ansatz which allows each qubit to have arbitrary rotation
    allocate(2, q)
    U(angles[0], angles[1], angles[2], 0, q[0])
    U(angles[0], angles[1], angles[2], 0, q[1])
    
    # create entanglement between the two qubits
    CX(q[0], q[1])



@cfunc
def cmain() -> None:
    res = vqe(
        # TODO: complete the missing argument
        HAMILTONIAN,
        False,
        [],
        optimizer=Optimizer.COBYLA,
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

#TODO: complete the line, use classical_execution_function
qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/bc8784d2-6c13-4f36-840e-ca36485adca6?version=0.42.0


In [7]:
execution = execute(qprog)
res = execution.result()
execution.open_in_ide()
vqe_result = res[0].value #TODO: complete the line

In [8]:
print(f"Optimal energy: {vqe_result.energy}")
print(f"Optimal parameters: {vqe_result.optimal_parameters}")
print(f"Eigenstate: {vqe_result.eigenstate}")

Optimal energy: -1.84520302734375
Optimal parameters: {'angles_0': 3.2017515243271615, 'angles_1': -5.448452277177368, 'angles_2': -3.027587294714585}
Eigenstate: {'11': (0.03125+0j), '10': (0.03125+0j), '01': (0.9990229601966113+0j)}


Does it similar to the `optimal energy` we calculated in class? \
Does it similar to the `total energy` we calculated in class? \
What can we learn about the provided  form this result Hamiltonian?

In [26]:
print(res[0].value)

energy=-1.84520302734375 time=0.4464535713195801 solution=None eigenstate={'11': (0.03125+0j), '10': (0.03125+0j), '01': (0.9990229601966113+0j)} reduced_probabilities={'11': 0.0009765625, '10': 0.0009765625, '01': 0.998046875} optimized_circuit_sample_results=ExecutionDetails(vendor_format_result={}, counts={'11': 2, '10': 2, '01': 2044}, counts_lsb_right=True, parsed_states={'11': {'q': 3.0}, '10': {'q': 2.0}, '01': {'q': 1.0}}, histogram=None, output_qubits_map={'q': (0, 1)}, state_vector=None, parsed_state_vector_states=None, physical_qubits_map={'q': (0, 1)}, num_shots=2048) intermediate_results=[VQEIntermediateData(utc_time=datetime.datetime(2024, 6, 2, 16, 17, 22, 758414, tzinfo=datetime.timezone.utc), iteration_number=1, parameters=[2.591176246955154, -5.62624493452543, -3.026748767095255], mean_all_solutions=-1.6016041015625, solutions=[], standard_deviation=0.378010120085899), VQEIntermediateData(utc_time=datetime.datetime(2024, 6, 2, 16, 17, 22, 773212, tzinfo=datetime.timez

In [27]:
print(f"Difference in Optimal energy: {vqe_result.energy - (-1.8657150931815518)}")

Difference in Optimal energy: 0.02051206583780174


## Discussion:

* The `optimal energy` from class was $-1.8657150931815518$ (Ha) which gives a difference of $\sim 0.0205$.
* Can't tell as directly as in class but if we could figure out the nuclear repulsion energy, then we would know a `total energy` for this case as well (by the difference with the optimal one).
* The provided `Hamiltonian` describes the molecular system defined in class quite well. I suppose we could trace back what each term means physically in the molecule itself.
In the following you can see my assumption on that.

\begin{aligned}
    - \sum_{i=1}^{2} \frac{\nabla_i^2}{2} &= -1.0523 \cdot (I \otimes I)\\
    - \frac{1}{|\mathbf{r}_1 - \mathbf{R}_A|} - \frac{1}{|\mathbf{r}_2 - \mathbf{R}_A|} &= + 0.3979 \cdot (I \otimes Z)\\
    - \frac{1}{|\mathbf{r}_1 - \mathbf{R}_B|} - \frac{1}{|\mathbf{r}_2 - \mathbf{R}_B|} &= - 0.3979 \cdot (Z \otimes I)\\
    \frac{1}{|\mathbf{r}_1 - \mathbf{r}_2|} &= - 0.0112 \cdot (Z \otimes Z)\\
    \frac{1}{|\mathbf{R}_A - \mathbf{R}_B|} &= + 0.1809 \cdot (X \otimes X)\\
\end{aligned}